In [1]:
import cv2
# from keras.models import Sequential
# from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
# from keras.optimizers import Adam
# from keras.preprocessing.image import ImageDataGenerator
import os
import random
import numpy as np
from io import BytesIO
import matplotlib.pyplot as plt
import tensorflow as tf
from ipywidgets import widgets
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
TRAIN_DIR="train"
TEST_DIR="test"
print(os.listdir())

['archive', 'archive.zip', 'Emotion_detection_with_CNN', 'Emotion_detection_with_CNN.zip', 'exp_det_project.ipynb', 'project_2.ipynb', 'test', 'train']


In [3]:
train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_data_generator.flow_from_directory(
    "train", 
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)

Found 24176 images belonging to 5 classes.


In [5]:
test_generator=test_data_generator.flow_from_directory(
    "test", 
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)

Found 6043 images belonging to 5 classes.


### build Nueral Network model

In [6]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(48,48,1)),
    tf.keras.layers.Conv2D(32, kernel_size=(3,3),activation="relu"),
    tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Conv2D(128,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(128,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024 ,activation="relu"),
    tf.keras.layers.Dense(5, activation="softmax")
    
])

In [7]:
#tf.keras.optimizers.Adam(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 44, 44, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 20, 20, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         5,125 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,343,557 (8.94 MB)

 Trainable params: 2,343,557 (8.94 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
cv2.ocl.setUseOpenCL(False)

In [11]:
emotion_model_info = model.fit(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=test_generator,
        validation_steps=7178 // 64)

Epoch 1/50
378/448 ━━━━━━━━━━━━━━━━━━━━ 13s 190ms/step - accuracy: 0.3178 - loss: 1.5387

c:\Users\Capiter\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


448/448 ━━━━━━━━━━━━━━━━━━━━ 82s 183ms/step - accuracy: 0.3267 - loss: 1.5241 - val_accuracy: 0.4966 - val_loss: 1.2181
Epoch 2/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 116s 258ms/step - accuracy: 0.5044 - loss: 1.2083 - val_accuracy: 0.5803 - val_loss: 1.0468
Epoch 3/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 114s 255ms/step - accuracy: 0.5765 - loss: 1.0445 - val_accuracy: 0.5903 - val_loss: 1.0057
Epoch 4/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 108s 240ms/step - accuracy: 0.6060 - loss: 0.9734 - val_accuracy: 0.6379 - val_loss: 0.9261
Epoch 5/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 124s 277ms/step - accuracy: 0.6403 - loss: 0.9192 - val_accuracy: 0.6394 - val_loss: 0.8832
Epoch 6/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 105s 234ms/step - accuracy: 0.6595 - loss: 0.8630 - val_accuracy: 0.6528 - val_loss: 0.8861
Epoch 7/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 116s 258ms/step - accuracy: 0.6796 - loss: 0.8248 - val_accuracy: 0.6634 - val_loss: 0.8580
Epoch 8/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 105s 233ms/step - accuracy: 0.7012 - loss: 0.767

In [34]:
import numpy as np

# Example of loading test data (assuming it's already preprocessed similarly to training data)
# Ensure test data shape is (batch_size, 300, 300, 1) for grayscale images
# You can use test_dataset or a single test image reshaped as (1, 300, 300, 1)

# Predict using a dataset
predictions = model.predict(test_generator)

#

# Output the prediction (for classification, this could be probabilities or class indices)
print(predictions[0])
print(np.argmax(predictions[0]))

95/95 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step
[6.7665486e-04 9.9598372e-01 2.1798650e-03 1.0239135e-03 1.3580364e-04]
1


In [29]:
test_generator[4]

(array([[[[0.35686275],
          [0.18823531],
          [0.16862746],
          ...,
          [0.32156864],
          [0.34509805],
          [0.3019608 ]],
 
         [[0.24705884],
          [0.09803922],
          [0.18823531],
          ...,
          [0.38823533],
          [0.27058825],
          [0.24313727]],
 
         [[0.14901961],
          [0.07058824],
          [0.1764706 ],
          ...,
          [0.37254903],
          [0.2509804 ],
          [0.28235295]],
 
         ...,
 
         [[0.9921569 ],
          [0.94117653],
          [0.9294118 ],
          ...,
          [1.        ],
          [1.        ],
          [1.        ]],
 
         [[0.9568628 ],
          [0.9058824 ],
          [0.9294118 ],
          ...,
          [1.        ],
          [1.        ],
          [1.        ]],
 
         [[0.93725497],
          [0.93725497],
          [0.9294118 ],
          ...,
          [1.        ],
          [1.        ],
          [1.        ]]],
 
 
        [

In [36]:
model_json = model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
model.save_weights('emotion_model.weights.h5')
